In [1]:
using Revise

In [2]:
using QuantumStates
using UnitsToValue
using InfiniteArrays
using HalfIntegers

[ Info: Precompiling QuantumStates [17f0441f-15e0-42ae-a101-302633ff8f0f]


In [3]:
Base.@kwdef struct VibrationalState <: BasisState
    E::Float64 = 0.0
    v1::HalfInt
    v2::HalfInt
    v3::HalfInt
    constraints = (
        v1 = 0:∞,
        v2 = 0:∞,
        v3 = 0:∞
        )
end
;

In [4]:
function unpack(state::VibrationalState)
    return state.v1, state.v2, state.v3
end

@inline function I(state::BasisState, state′::BasisState)
    return δ(unpack(state), unpack(state′))
end

function v1(state::VibrationalState, state′::VibrationalState)
    return I(state, state′) * state.v1
end

function v2(state::VibrationalState, state′::VibrationalState)
    return I(state, state′) * state.v2
end

function v3(state::VibrationalState, state′::VibrationalState)
    return I(state, state′) * state.v3
end
;

In [ ]:
@make_operator 

In [6]:
H_operator = :(
    T * I +
    ωe1 * v1 +
    ωe2 * v2 +
    ωe3 * v3
    )
;

In [27]:
QN_bounds = (v1 = 0:3, v2 = 0:3, v3 = 0)
basis = enumerate_states(VibrationalState, QN_bounds)
;

In [40]:
esc(4)

:($(Expr(:escape, 4)))

┌ Warning: G:\My Drive\github\UnitsToValue\src is not an existing directory, Revise is not watching
└ @ Revise C:\Users\Christian\.julia\packages\Revise\Kw99o\src\packagedef.jl:574
┌ Warning: G:\My Drive\github\QuantumStates\src is not an existing directory, Revise is not watching
└ @ Revise C:\Users\Christian\.julia\packages\Revise\Kw99o\src\packagedef.jl:574


In [33]:
unpack_operator(H_operator, basis)

LoadError: MethodError: objects of type Expr are not callable

In [20]:
X_parameters = @params begin
    T = 0.0
    ωe1 = 609
    ωe2 = 353
    ωe3 = 3847
end
X_Hamiltonian = Hamiltonian(basis=basis, operator=H_operator, parameters=X_parameters)

LoadError: UndefVarError: `ωe1` not defined

### X state

In [7]:
X = state(0, 621, 352.93, 0);

### A state

In [8]:
A = state(15997.78, 630.68, 361.36, 0);

### B state

In [9]:
B = state(18022.27, 600, 300, 0);

In [10]:
C = state(21896, 528, 312, 0);

In [11]:
D = state(28156.20, 674.0, 412, 0);

In [12]:
E_ = state(29879, 600, 300, 0);

### Create list of transition wavelengths

In [13]:
A000_freq = 478.600015
X0200_to_A000 = 457.954463
X0200_freq = A000_freq - X0200_to_A000
X0200_to_A100 = 476.582353
X0220_to_A100 = 475.852270
X0220_freq = X0200_freq + (X0200_to_A100 - X0220_to_A100)
;

In [14]:
struct transition
    E::Float64
    v1::Int
    v2::Int
    v1′::Int
    v2′::Int
end

In [15]:
transitions = []
for v1′ ∈ 0:3, v2′ ∈ 0:3
    transition_E = 1e7 ./ (E(D, v1′, v2′, 0) - 1e10 * X0200_freq / c)
    push!(transitions, transition(transition_E, 0, 2, v1′, v2′))
end

In [17]:
1e-9 * c / (X0220_freq - X0200_freq)

0.4106279121688702

In [26]:
comb_wavelength = 1 / (1 / 630.0 + 1 / 778.030)

348.11680148860466

In [27]:
[transition for transition in transitions if abs(transition.E - comb_wavelength) < 10]

9-element Vector{transition}:
 transition(353.4625731118179, 0, 2, 0, 2)
 transition(348.38910176759646, 0, 2, 0, 3)
 transition(355.3465989600422, 0, 2, 1, 0)
 transition(350.21929052832917, 0, 2, 1, 1)
 transition(345.23784170650663, 0, 2, 1, 2)
 transition(340.3961157490556, 0, 2, 1, 3)
 transition(347.0349858491749, 0, 2, 2, 0)
 transition(342.14307837268285, 0, 2, 2, 1)
 transition(339.1033059294837, 0, 2, 3, 0)

In [345]:
I1 = 10 * 100
Ω1 = (3e-30) * sqrt(2 * 377 * I1) / h
Ω2 = 25e9
det = 0.1 * 1e12
;

In [346]:
Ω1*Ω2 / 2det

491429.147020706